In [126]:
import torch
import SimpleITK as sitk
import pandas
import glob, os
import numpy
import tqdm
import pylidc
from torch.autograd import Variable

In [127]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [128]:
def model(t_u, w1, w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b

In [129]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [130]:
w1 = torch.ones(())
w2 = torch.ones(())
b = torch.zeros(()) 
t_p = model(t_u, w1, w2, b)
t_p

tensor([1310.1901, 3180.7100, 3445.4399, 6789.5103, 3225.9900, 2440.1101,
        1183.1101,  497.0399, 2390.9600, 3708.5601, 4746.9600])

In [131]:
loss = loss_fn(t_p, t_c)
loss

tensor(11709471.)

In [132]:
delta = 0.1
loss_rate_of_change_w = \
(loss_fn(model(t_u, w1 + delta, w2 + delta, b), t_c) -
loss_fn(model(t_u, w1- delta, w2 - delta, b), t_c)) / (2.0 * delta)

In [133]:
learning_rate = 1e-4
w = w - (learning_rate * loss_rate_of_change_w)

In [134]:
loss_rate_of_change_b = \
(loss_fn(model(t_u, w1, w2, b + delta), t_c) -
loss_fn(model(t_u, w1, w2, b - delta), t_c)) / (2.0 * delta)
b = b - (learning_rate * loss_rate_of_change_b)

In [135]:
def model(t_u, w1, w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b

In [136]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [137]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)
    return dsq_diffs

In [138]:
def dmodel_dw(t_u, w1, w2, b):
    return t_u

In [139]:
def dmodel_db(t_u, w1, w2, b):
    return 1.0

In [140]:
def grad_fn(t_u, t_c, t_p, w1, w2, b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw1 = dloss_dtp * dmodel_dw(t_u, w1, w2, b)
    dloss_dw2 = dloss_dtp * dmodel_dw(t_u, w1, w2, b)
    dloss_db = dloss_dtp * dmodel_db(t_u, w1, w2, b)
    return torch.stack([dloss_dw1.sum(), dloss_dw2.sum(), dloss_db.sum()])

In [141]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        w1, w2, b = params
        t_p = model(t_u, w1, w2, b)
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w1, w2, b)
        params = params - learning_rate * grad 
        print('Epoch %d, Loss %f' % (epoch, float(loss)))
        
    return params

In [149]:
t_un = 0.1 * t_u
training_loop(
n_epochs = 5000,
learning_rate = 1e-1,
params = torch.tensor([1.0, 1.0, 0.0]),
t_u = t_un,
t_c = t_c)

Epoch 1, Loss 675.794373
Epoch 2, Loss 1150713.125000
Epoch 3, Loss 1912229376.000000
Epoch 4, Loss 3180625920000.000000
Epoch 5, Loss 5290245973082112.000000
Epoch 6, Loss 8799122176742522880.000000
Epoch 7, Loss 14635340953666459795456.000000
Epoch 8, Loss 24342572510712119391420416.000000
Epoch 9, Loss 40488342408838791954473943040.000000
Epoch 10, Loss 67343169260303412050357244657664.000000
Epoch 11, Loss 112010092308661749785316455683719168.000000
Epoch 12, Loss inf
Epoch 13, Loss inf
Epoch 14, Loss inf
Epoch 15, Loss inf
Epoch 16, Loss inf
Epoch 17, Loss inf
Epoch 18, Loss inf
Epoch 19, Loss inf
Epoch 20, Loss inf
Epoch 21, Loss inf
Epoch 22, Loss inf
Epoch 23, Loss inf
Epoch 24, Loss inf
Epoch 25, Loss inf
Epoch 26, Loss nan
Epoch 27, Loss nan
Epoch 28, Loss nan
Epoch 29, Loss nan
Epoch 30, Loss nan
Epoch 31, Loss nan
Epoch 32, Loss nan
Epoch 33, Loss nan
Epoch 34, Loss nan
Epoch 35, Loss nan
Epoch 36, Loss nan
Epoch 37, Loss nan
Epoch 38, Loss nan
Epoch 39, Loss nan
Epoch 40, 

tensor([nan, nan, nan])

In [150]:
t_un = 0.1 * t_u
training_loop(
n_epochs = 5000,
learning_rate = 1e-2,
params = torch.tensor([1.0, 1.0, 0.0]),
t_u = t_un,
t_c = t_c)

Epoch 1, Loss 675.794373
Epoch 2, Loss 7072.701172
Epoch 3, Loss 70292.179688
Epoch 4, Loss 713358.250000
Epoch 5, Loss 7195382.500000
Epoch 6, Loss 72719712.000000
Epoch 7, Loss 734488512.000000
Epoch 8, Loss 7419956224.000000
Epoch 9, Loss 74953383936.000000
Epoch 10, Loss 757163163648.000000
Epoch 11, Loss 7648655179776.000000
Epoch 12, Loss 77264792322048.000000
Epoch 13, Loss 780509038772224.000000
Epoch 14, Loss 7884501782888448.000000
Epoch 15, Loss 79647196386951168.000000
Epoch 16, Loss 804575605084389376.000000
Epoch 17, Loss 8127615791043444736.000000
Epoch 18, Loss 82103119297382449152.000000
Epoch 19, Loss 829384877701387517952.000000
Epoch 20, Loss 8378232160281962741760.000000
Epoch 21, Loss 84634751681289246474240.000000
Epoch 22, Loss 854958597458808457396224.000000
Epoch 23, Loss 8636575343262551936860160.000000
Epoch 24, Loss 87244457169047646218747904.000000
Epoch 25, Loss 881321163385489572134649856.000000
Epoch 26, Loss 8902882732536723748516003840.000000
Epoch 27

tensor([nan, nan, nan])

In [151]:
t_un = 0.1 * t_u
training_loop(
n_epochs = 5000,
learning_rate = 1e-3,
params = torch.tensor([1.0, 1.0, 0.0]),
t_u = t_un,
t_c = t_c)

Epoch 1, Loss 675.794373
Epoch 2, Loss 229.416977
Epoch 3, Loss 80.701340
Epoch 4, Loss 31.793343
Epoch 5, Loss 16.086189
Epoch 6, Loss 11.267120
Epoch 7, Loss 9.926227
Epoch 8, Loss 9.640760
Epoch 9, Loss 9.641191
Epoch 10, Loss 9.696720
Epoch 11, Loss 9.746609
Epoch 12, Loss 9.780423
Epoch 13, Loss 9.800562
Epoch 14, Loss 9.811265
Epoch 15, Loss 9.815983
Epoch 16, Loss 9.817051
Epoch 17, Loss 9.815939
Epoch 18, Loss 9.813540
Epoch 19, Loss 9.810383
Epoch 20, Loss 9.806787
Epoch 21, Loss 9.802934
Epoch 22, Loss 9.798934
Epoch 23, Loss 9.794848
Epoch 24, Loss 9.790711
Epoch 25, Loss 9.786548
Epoch 26, Loss 9.782373
Epoch 27, Loss 9.778189
Epoch 28, Loss 9.774001
Epoch 29, Loss 9.769810
Epoch 30, Loss 9.765622
Epoch 31, Loss 9.761433
Epoch 32, Loss 9.757249
Epoch 33, Loss 9.753065
Epoch 34, Loss 9.748879
Epoch 35, Loss 9.744701
Epoch 36, Loss 9.740520
Epoch 37, Loss 9.736349
Epoch 38, Loss 9.732174
Epoch 39, Loss 9.728004
Epoch 40, Loss 9.723838
Epoch 41, Loss 9.719671
Epoch 42, Loss 9.

tensor([ 0.4414,  0.4414, -4.5622])

In [152]:
t_un = 0.1 * t_u
training_loop(
n_epochs = 5000,
learning_rate = 1e-4,
params = torch.tensor([1.0, 1.0, 0.0]),
t_u = t_un,
t_c = t_c)

Epoch 1, Loss 675.794373
Epoch 2, Loss 620.295959
Epoch 3, Loss 569.380798
Epoch 4, Loss 522.672058
Epoch 5, Loss 479.823914
Epoch 6, Loss 440.518829
Epoch 7, Loss 404.465515
Epoch 8, Loss 371.396210
Epoch 9, Loss 341.065399
Epoch 10, Loss 313.247681
Epoch 11, Loss 287.736115
Epoch 12, Loss 264.340637
Epoch 13, Loss 242.887100
Epoch 14, Loss 223.215317
Epoch 15, Loss 205.178360
Epoch 16, Loss 188.641556
Epoch 17, Loss 173.481049
Epoch 18, Loss 159.583267
Epoch 19, Loss 146.843979
Epoch 20, Loss 135.167450
Epoch 21, Loss 124.465889
Epoch 22, Loss 114.658760
Epoch 23, Loss 105.672043
Epoch 24, Loss 97.437843
Epoch 25, Loss 89.893944
Epoch 26, Loss 82.983078
Epoch 27, Loss 76.652832
Epoch 28, Loss 70.855072
Epoch 29, Loss 65.545586
Epoch 30, Loss 60.683895
Epoch 31, Loss 56.232750
Epoch 32, Loss 52.158024
Epoch 33, Loss 48.428452
Epoch 34, Loss 45.015255
Epoch 35, Loss 41.892082
Epoch 36, Loss 39.034767
Epoch 37, Loss 36.421124
Epoch 38, Loss 34.030796
Epoch 39, Loss 31.845110
Epoch 40, L

tensor([ 0.3497,  0.3497, -0.8786])